# python私房手册-redis全攻略

以下内容全部基于redis6.2.6,centos7版本，如果有新的版本，造成内容不同，会额外声明。

## 安装

安装redis很简单，在linux下依次运行以下几个命令：
```shell
$ yum install gcc gcc-c++ -y  # 需要先安装gcc编译库，-y表示自动选择，所有安装选项全部选择y
$ wget http://download.redis.io/releases/redis-6.2.6.tar.gz  #这里指定你的版本
$ tar xzf redis-6.2.6.tar.gz  # 解压缩
$ ln -s redis-6.2.6 redis  # 创建软链接，这一步不是必需的，但是后期升级什么的比较简单，直接修改软连接就行了
$ make  # 开始编译
$ make install  # 把一些执行文件放到/usr/bin文件夹中，这样就可以直接运行指令了
```
但是安装完成以后有几点要注意，直接运行redis-server会报错，我的出现以下几个错误：
1. `Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf`:这个是因为没有使用配置文件启动，直接`redis-server /path/to/redis.conf`使用配置文件即可。
2. `WARNING: The TCP backlog setting of 511 cannot be enforced because /proc/sys/net/core/somaxconn is set to the lower value of 128.`: 这个是因为linux系统默认的连接数太少了，默认才128，最少512以上，`net.core.somaxconn = 1024`添加到`/etc/sysctl.conf`中，然后执行`sysctl -p`生效配置即可。
3. `WARNING overcommit_memory is set to 0! Background save may fail under low memory condition. To fix this issue add 'vm.overcommit_memory = 1' to /etc/sysctl.conf and then reboot or run the command 'sysctl vm.overcommit_memory=1' for this to take effect.`：`overcommit_memory`是linux内核分配的一种策略，解释如下：
 - overcommit_memory=0， 表示内核将检查是否有足够的可用内存供应用进程使用；如果有足够的可用内存，内存申请允许；否则，内存申请失败，并把错误返回给应用进程。
 - overcommit_memory=1， 表示内核允许分配所有的物理内存，而不管当前的内存状态如何。
 - overcommit_memory=2， 表示内核允许分配超过所有物理内存和交换空间总和的内存
 
 只需要将`vm.overcommit_memory = 1`添加到`/etc/sysctl.conf`中，然后执行`sysctl -p`生效配置。

还有一些常见问题如下：
1. **redis desktop manager无法连接redis服务器？**   
安装完以后，如果直接桌面客户端连接，有可能失败，连接不上。这是因为`redis`配置文件默认只运行本机连接，需要修改配置文件，修改第75行的`bind 127.0.0.1`改为`0.0.0.0`，允许所有ip。
2. **如何设置密码？**   
如果要设置一个密码，有两种方式，一种是临时性的，不需要重启redis，在redis-cli中，输入：`config set requirepass yourpass`，再使用redis-cli查询，会提示需要密码，直接输入`auth yourpass`即可。但是redis重启以后，就失效了。如果要永久有效，可以配置文件，在901行，修改`requirepass yourpass`即可。
3. **如何后台启动redis？**  
默认情况下，启动redis以后不会返回命令行，此时可以修改配置，257行`daemonize no`修改为`yes`。